In [15]:
%%writefile swap_key_value.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    word, count = unicode(line.strip()).split('\t', 1)

    print "%s\t%s" % (count, word)

Overwriting swap_key_value.py


In [16]:
%%writefile reducer_identity.py

import sys

for line in sys.stdin:
    count, word = line.strip().split('\t', 1)
    print "%s\t%s" % (word, count)

Overwriting reducer_identity.py


In [17]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [18]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer.py


In [19]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_DIR_TOP=${OUT_DIR}_top
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} ${OUT_DIR_TOP} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Top words" \
    -D mapred.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options=-nr \
    -D mapreduce.job.reduces=1 \
    -files swap_key_value.py,reducer_identity.py \
    -mapper "python swap_key_value.py" \
    -reducer "python reducer_identity.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_TOP} > /dev/null
    
hdfs dfs -cat ${OUT_DIR_TOP}/part-00000 | head -n 7 | tail -n 1

rm: `wordcount_result_1525011469916544': No such file or directory
rm: `wordcount_result_1525011469916544_top': No such file or directory
18/04/29 14:17:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/29 14:17:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/29 14:17:53 ERROR streaming.StreamJob: Error Launching job : Cannot create directory /tmp/hadoop-yarn/staging/jovyan/.staging. Name node is in safe mode.
Resources are low on NN. Please add or free up more resources then turn off safe mode manually. NOTE:  If you turn off safe mode before adding resources, the NN will immediately return to safe mode. Use "hdfs dfsadmin -safemode leave" to turn safe mode off.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1372)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1360)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.mkdirs(FSName